In [8]:
# Arreglar:

# Millorar:
# - temps de Ei
# - temps long galois group (>30 min)
# - afegir enters quan continue????

# Fer a part:
# - error: eval('(1/2)*1*t**3 - t + 1'.replace('1/2', 'm'), {'t': CC(2), 'm':CC(1/2)})

In [2]:
import pandas as pd
from sage.interfaces.gp import *
import json
import numpy as np
from datetime import datetime
CC = ComplexBallField(10000)
RR = RealBallField(10000)
RF = RealField(10000)

In [3]:
def lfunction_value(cf, cWf, N, prc1 = None, prc2 = None):
    """
    cf: Coefficients of the modular form
    cWf: Coefficients of W(f)
    N: level of the modular form
    """
    if prc1 == None:
        prc1 = [0] + [-RR(Ei(RF(-2*pi*n)/RF(sqrt(N)))) for n in range(1, 3000)] # CC = 2000, Precision: 200 - e-547
    if prc2 == None:
        prc2 = [-RR(integral(exp(-2*pi*n*x), x)(x = RR(1)/RR(sqrt(N)))) for n in range(3000)] # CC = 2000, Precision: 10.000 - e-210
    
    return sum([CC(cf[i])*RR(prc1[i]) for i in range(min(len(cf), len(prc1)))]) + \
        RR(N**(1/2))*CC(0, 1)*sum([CC(cWf[i])*RR(prc2[i]) for i in range(min(len(cWf), len(prc2)))])

def evaluate(coef, a, b):
    """
    Evaluates modular form with coefficients coef at point a + bi
    """
    return sum([CC(coef[n])*exp(RR(2)*RR(pi)*RR(n)*CC(a, b)*CC(0, 1)) for n in range(len(coef))])

def evaluate_W(coef, a, b):
    """
    If coef are the coefficients of f, evaluates modular form W(f) at point a + bi
    """
    return RR(N**(-1/2))/CC(a, b)*sum([CC(coef[n])*exp(RR(-1)*RR(2)*RR(pi)*RR(n)*CC(0, 1)/(RR(N)*CC(a, b))) for n in range(len(coef))])

In [4]:
df = pd.read_csv('wt1.txt', sep=':', names=['label', 'character', 'coef. polynomial'], dtype={'character': str})
results = pd.read_csv('res.txt', sep=':', index_col=0)
results_exp = pd.read_csv('res_exp.txt', sep=':', index_col=0)
#results = df.copy()
#results['Stark unit, b=1'] = '-'
#results_exp  = df.copy()
#results_exp['Stark unit, b=1'] = '-'
#results_exp['L-value'] = '-'
#results_exp['precision L-value'] = '-'
#results_exp['first coefficients'] = '-'

#pos = 41
#results.loc[pos:pos, 'Stark unit, b=1'] = '-'
#results_exp.loc[pos:pos, 'Stark unit, b=1'] = '-'

#results.to_csv("res.txt", sep=':')
#results_exp.to_csv("res_exp.txt", sep=':')

In [5]:
def find_polynomial(value, l, prec):
    R.<x> = ZZ[]
    
    for deg in l:
        val_str = str(RF(value.real()))
        closest_pol = gp('algdep(' + val_str[:prec] + ', ' + str(deg) + ')')
        factors = str(R(str(closest_pol)).factor()).split(' * ')
        deg_factors = [int(factors[i].split('^')[1].split(' ')[0]) if '^' in factors[i] else 1 for i in range(len(factors))]

        # Find max degree pol
        arg_max = 0
        val_max = deg_factors[0]
        for i in range(len(deg_factors)):
            if deg_factors[i] > val_max:
                val_max = deg_factors[i]
                arg_max = i
                
        irr_poly = str(R(str(R(str(closest_pol)).factor()).split(' * ')[arg_max]))
        if irr_poly.split('^')[0] == 'x' and irr_poly[-2:] == ' 1':
            return irr_poly
    
    return '-'

def presumed_precision(N, n):
    a = 1e-6; b = RR(1/sqrt(N)); # a + bi is the point of evaluation
    p_eval_f = -RealField(15)(log(abs(exp(RR(2)*RR(pi)*RR(n)*CC(a, b)*CC(0, 1)).real()), 10))
    p_eval_Wf = -RealField(15)(log(abs((RR(N**(-1/2))/CC(a, b)*exp(RR(-1)*RR(2)*RR(pi)*RR(n)*CC(0, 1)/(RR(N)*CC(a, b)))).real()), 10))
    p_t1_L = -RealField(15)(log(abs(-RR(integral(exp(-2*pi*n*x), x)(x = RR(1/sqrt(N))))), 10)) # First term of L'(f, 0)
    p_t2_L = -RealField(15)(log(abs(-RR(Ei(RF(-2*pi*n/sqrt(N))))), 10)) # Second term of L'(f, 0)

    return min(p_eval_f, p_eval_Wf, p_t1_L, p_t2_L)

def num(c):
    if c == 'a':
        return 0
    if c == 'b':
        return 1
    if c == 'c':
        return 2
    if c == 'd':
        return 3
    if c == 'e':
        return 4
    if c == 'f':
        return 5
    if c == 'g':
        return 6
    if c == 'h':
        return 7
    if c == 'i':
        return 8
    
    return '-'

In [12]:
coefs = 5000
b = 1
R.<x> = ZZ[]

prev = -1 # Previous level
for index, row in df.loc[14976:14976].iterrows():
    
    # Start
    N = int(row['label'].split('.')[0])
    print('STARTING', index,', N = ', N, ', Label ', row['label'], ', Time:', datetime.now().time())
    
    try:
    
        # Check that it is not done
        results = pd.read_csv('res.txt', sep=':', index_col=0)
        results_exp = pd.read_csv('res_exp.txt', sep=':', index_col=0)
        if results.index.dtype != 'int64' or results_exp.index.dtype != 'int64':
            print('OJO!')
            break
        # if results.loc[index, 'Stark unit, b=1'] != '-' or len(json.loads(row['coef. polynomial']))>2 or results_exp.loc[index, 'Stark unit, b=1'] == 'error' or results_exp.loc[index, 'L-value'] != '-':
        #     print()
        #     continue

        # Compute expected precision
        pres_prec = presumed_precision(N, coefs)
        print('Presumed precision: ', pres_prec, 'digits')

        # Get coefficients
        print('Starting coefficients, Time:', datetime.now().time())
        level, charid = row['character'].split('.')
        # qs = gp('[mfcoefs(f,5001)|f<-mfeigenbasis(mfinit(['+level+',1,Mod('+charid+','+level+')],0))]')

        # Choose mf sorted by trace
        # arg = 0
        # if len(qs) > 1:
        #     lab = row['label'].split('.')[-1]
        #     gp('qs = [mfcoefs(f,5001)|f<-mfeigenbasis(mfinit(['+level+',1,Mod('+charid+','+level+')],0))];');
        #     traces = []
        #     for i in range(1, len(qs)+1):
        #         if str(qs[i][2])[0] != 'M':
        #             ts_i = gp('[trace(Mod(qs['+str(i)+'][j], y))|j<-[1..20]]')
        #         else:
        #             ts_i = gp('[trace(trace(qs['+str(i)+'][j]))|j<-[1..20]]')
        #         traces = traces + [list(ts_i)]
        #     traces_sorted = traces.copy()
        #     traces_sorted.sort()
        #     arg = -1
        #     cand = 0
        #     while arg == -1:
        #         if traces_sorted[num(lab)] == traces[cand]:
        #             arg = cand
        #         cand = cand + 1
        # gp('qs = [mfcoefs(f[1],5001)|f<-mfeigenbasis(mfinit(['+level+',1,Mod('+charid+','+level+')],0))];');
        # coefs_str = [(str(el).split('('))[1].split(',')[0] if str(el)[0] == 'M' else str(el) for el in qs]

        # Convert coefficients to readable
        # coefs_str = [(str(el).split('('))[1].split(',')[0] if str(el)[0] == 'M' else str(el) for el in qs[arg+1]]
        # print('First coefficients :', coefs_str[:10])
        # if coefs_str[1] == 'Mod':
        #     results_exp = pd.read_csv('res_exp.txt', sep=':', index_col=0)
        #     results_exp.loc[index, 'Stark unit, b=1'] = 'Mod problem'
        #     results_exp.to_csv("res_exp.txt", sep=':')
        #     print('Mod problem')
        #     print('End. Time:', datetime.now().time())
        #     print('')
        #     continue

        # var = str(qs[arg+1])[8] if str(qs[arg+1])[8].islower() else str(qs[arg+1])[11] # variable used by PARI, usually 'y' or 't'
        # print('The variable used is', var)

        # Get Galois group
        print('Starting Galois group, Time:', datetime.now().time())
        poly_l = json.loads(row['coef. polynomial'])
        if var == 't':
            M.<t> = NumberField(sum(coef*x**i for i, coef in enumerate(poly_l)))
        else: 
            M.<y> = NumberField(sum(coef*x**i for i, coef in enumerate(poly_l)))
        G = M.galois_group()
        print('The Galois group is', G)

        # Get correct format for cf
        # if len(poly_l) > 2:
        #     cf = [sage_eval(co.replace('^', '**')) if var not in co else eval(co.replace('^', '**')) for co in coefs_str]
        # else:
        #     cf = [sage_eval(co.replace('^', '**')) for co in coefs_str]
        cf = [0, 1, 2, -1, 3, 0, -2, -1, 4, 0, 0, -1, -3, 2, -2, 0, 5, 0, 0, 0, 0, 1, -2, 0, -4, 1, 4, 1, -3, 0, 0, 0, 6, 1, 0, 0, 0, 2, 0, -2, 0, -1, 2, 0, -3, 0, 0, -1, -5, 0, 2, 0, 6, -1, 2, 0, -4, 0, 0, -1, 0, -1, 0, 0, 7, 0, 2, -1, 0, 0, 0, -1, 0, -1, 4, -1, 0, 1, -4, 0, 0, -1, -2, 0, 3, 0, 0, 0, -4, -1, 0, -2, 0, 0, -2, 0, -6, -1, 0, 0, 3, 0, 0, 0, 8, 0, -2, 0, 3, 2, 0, -2, -5, 0, 0, 0, 0, 0, -2, 0, 0, 0, -2, 1, 0, 0, 0, 2, 8, 0, 0, 0, 3, 0, -2, 0, 0, -1, 0, 0, 0, 1, -2, -2, 0, 0, -2, 0, 6, -1, -2, 0, 0, 0, 2, 0, -6, -1, 0, 1, 0, 0, -2, -1, -3, 0, 0, 2, 4, 3, 0, 0, 0, 0, 0, -1, -5, 1, -2, -1, 0, 0, -4, 1, 0, 0, 0, 0, -3, -1, 0, 0, -7, -1, -2, 0, 0, 0, 0, -1, 4, 1, 0, 0, 0, 0, 0, 0, 10, 0, 0, 2, -3, 1, 0, 0, 4, 0, 4, 1, 0, 0, -4, 0, -6, 0, 0, 2, 0, 0, 0, -1, 0, 0, 0, 0, -3, 0, 0, 0, 0, 0, 0, 0, -3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 9, 0, 0, -2, 0, 0, 0, -1, 4, 0, 0, 1, -3, 0, 0, 0, 0, 2, -2, -1, 0, -1, 0, 0, 0, 2, 2, 0, -3, 0, -4, 1, 0, 1, 0, 1, -3, -1, 0, 0, 8, -1, -2, 0, -3, 0, 0, 0, 0, 0, 0, -1, 3, 0, 0, 2, -8, -1, -2, 0, 0, 0, 2, 0, 0, 0, 0, 0, -3, 2, -2, -2, -4, 1, 0, -1, 0, 0, 4, 0, 5, -1, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, -2, 2, -6, 0, 2, 0, -3, 0, -2, 0, 0, 1, 0, 0, -6, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, -4, 0, -2, -1, 0, -2, 0, 0, -8, 0, -2, 0, -3, -1, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 5, 0, 2, 0, 0, 0, 0, -2, 0, -1, 0, 1, 0, 1, 0, 0, 12, 0, 0, 2, 0, 0, 4, 0, -4, 0, 2, 1, 0, 2, 0, 2, 5, 0, 0, 0, 6, 0, 2, 0, 0, 0, 0, -1, -6, 0, 0, 1, -7, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 2, -2, -1, 0, 0, 0, -1, 0, 1, 0, 1, -4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, -1, -4, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, -2, 0, 0, 0, 0, 0, -3, 6, -1, 0, 1, 10, 0, 0, 0, 0, 1, -4, 0, 0, 0, 0, 2, 0, 1, -2, 0, 5, 1, 0, 0, 0, -2, 2, 0, -4, 1, 0, 0, 0, -1, 0, 0, 0, 0, 4, -1, -3, 0, -2, 0, 0, 0, -2, 0, 0, -1, 0, 0, 0, 0, 4, 2, 3, 0, 0, 1, -4, -1, 0, 0, -6, 0, 2, 0, 0, 0, 2, 1, 0, 0, 2, 1, -4, 0, -2, -1, 0, 0, 0, 0, 10, 0, -2, 0, -3, 1, 0, 0, -4, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, -2, 0, 4, 0, 0, -1, 0, 0, 4, 1, -10, 1, -2, 0, -3, 0, 0, -1, 0, -2, 0, 0, 3, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -4, 1, 4, 0, -3, 2, -4, 0, -5, 0, 2, -1, 0, 0, -2, 0, 0, 0, 0, 0, 6, 0, 0, 1, 6, -1, -2, 1, 9, -1, 0, 1, 0, -2, 0, 0, 0, 0, 2, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -3, 0, 4, 0, -7, 0, 0, 0, 3, -1, 0, 0, -4, 0, 0, 0, -3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, -1, -8, 1, 0, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, -5, 0, 0, 0, -3, 0, -2, 0, 0, 0, -4, -2, 0, 0, 0, -2, -9, 0, 0, 0, -3, 2, 0, 0, -4, 2, -2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -2, 0, 0, -3, 0, 0, 0, 6, 0, 0, 1, 3, 0, 0, 0, 0, -1, 0, 0, 0, 0, -4, 0, 0, 0, -2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 1, 14, 0, 0, 0, 0, 0, 4, 2, 0, 1, 0, -2, 6, 0, 0, 0, -5, 0, 0, 0, 3, 0, 2, 0, 0, -1, 4, 0, 0, -1, 4, 0, 6, 0, 0, -1, 0, 0, 0, -2, 8, 0, 0, 0, 3, 0, 0, 1, 0, -1, 0, 0, 0, 0, -2, 2, -8, -2, 0, 1, 0, 0, 2, 0, -8, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, -2, -3, 2, -2, 0, 0, 0, 0, 0, 0, -2, -2, 0, 0, 0, 2, 1, 0, 0, 2, 0, -5, 0, 0, -1, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 8, 0, 0, 0, 0, -1, 0, 0, 0, 2, 0, 0, -2, -2, -5, 2, 2, 1, 0, 0, 0, 2, 4, 0, 0, -1, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 2, 0, 2, -4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, -6, 0, 8, 0, -2, 0, 0, 2, 2, 0, 11, -1, 0, 0, 0, -1, 0, 0, 0, 0, 2, 0, -6, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 1, 0, 2, 2, 0, -3, -2, 0, 0, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, -4, 1, 3, 0, 0, 0, -5, 0, 2, 0, 0, 0, 0, 0, 0, 0, -2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, -2, 0, -4, -1, 0, 1, -3, 0, 0, 0, 0, 0, 0, -1, -3, -1, 0, 0, 0, -1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 4, 0, 4, 2, 0, 0, 0, 0, 2, 0, -5, 1, -2, 0, 0, 1, 0, 0, -8, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 3, -2, 2, 0, 0, 0, 0, -1, 3, 0, 2, 1, -5, -2, 0, -1, -3, 0, -2, -1, 0, 0, 0, 0, 0, 0, 0, -3, 12, 0, 0, -1, -3, 0, 0, 0, -4, 0, 2, 0, 0, 0, 0, -2, -5, -1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, -4, 0, 0, 0, 0, 1, -3, 0, 0, 2, 5, 0, 0, 0, 0, 0, -2, -1, 0, 0, 0, 0, 6, 0, 2, 0, -12, -1, 2, 0, -3, 1, 0, 0, -4, -2, 0, 0, 0, -2, -2, 0, 0, 0, -4, 0, 0, -1, 0, 0, 4, 0, 0, 0, 0, 2, 0, -1, 0, -1, 0, 0, 0, 0, -2, 0, 0, -1, 0, 0, 0, -2, 0, 0, -5, -1, 2, 0, 6, 0, 0, 0, -4, 0, 4, 0, -6, 0, 0, 0, -6, 0, 0, 0, 3, 0, -2, -1, 0, 0, 0, 0, -3, -1, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 2, 0, 7, 0, -2, 0, -3, 0, 2, 1, 12, -1, -2, 0, 0, 0, 2, 0, 0, -1, -4, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, -1, 0, 0, 0, 0, -4, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, -3, -2, 0, -1, -4, 1, 0, 0, 6, 0, 0, -1, -8, 0, 0, 0, 0, 0, 0, 0, 4, 4, -2, 0, 0, 0, 0, -1, -5, 0, 0, -1, 0, 0, 0, -1, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 3, 0, 0, -1, 0, 0, 0, 0, 0, 0, -2, 0, -10, 0, 2, 0, 0, 1, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, -1, 0, -2, 0, 0, 0, 2, 0, 0, 3, 0, 0, -1, 0, 0, 0, -1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, -6, 0, 0, 1, 0, 0, 0, 2, -4, 0, 0, 0, -3, -2, 0, 0, 0, 0, 0, 0, -6, -1, -4, 1, 0, 0, 0, 0, 0, -1, -4, 0, -10, 0, 0, 0, 0, 0, 0, -1, -4, 0, 4, 0, 0, 2, 0, -1, -5, 2, 4, 0, -3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, -2, 0, -1, 0, 0, 0, 0, 0, 0, 2, -1, 0, 0, 0, -1, 0, 0, -4, -1, 0, -2, 0, 0, -4, -1, 0, 0, 0, 0, 0, 2, 7, -1, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 0, -1, 0, 0, 0, 0, -2, 0, 0, -1, 0, 1, 0, 0, 0, -1, -6, 0, 0, 0, 0, 0, 0, 0, -3, -1, 0, 1, 0, 1, 0, 0, 3, 0, 0, -1, 0, 0, 2, 4, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 16, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, -1, 6, 0, 4, 0, 0, 0, 2, 0, 0, 0, -4, 0, 8, -2, 0, 0, 0, 0, 0, 0, -6, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 3, 2, 0, 0, 0, 0, -2, 0, 6, 0, 0, 0, 0, 0, -2, -1, 6, 0, 0, 1, 7, 0, 0, 0, 0, -1, -2, 0, 0, 0, 0, -2, 0, -1, -4, 0, 10, 0, 0, 0, 0, -1, 0, 0, 4, -1, 0, 0, 0, 0, 2, -1, 0, 1, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3, 0, 4, -1, -10, -1, -4, 0, 0, -2, 2, -1, 0, 0, 0, -1, 3, 0, 0, 0, -9, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 2, 0, 0, 0, -2, 0, 0, 0, 1, 2, 0, 6, 0, -4, -1, -4, 1, 4, 0, -3, 0, 0, 0, 0, 1, 0, -3, 0, 0, 0, 0, 0, 0, -4, 2, -3, -1, 0, 0, 0, 0, 0, -1, 3, 0, 2, -1, 0, 0, 0, 0, 3, 0, 0, 0, -6, 2, 0, 0, 0, 1, -2, 0, 0, 0, -4, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 2, 0, 0, 0, -1, 2, 0, 0, 0, 2, 0, 12, 1, 0, 1, 0, 1, 0, 0, 0, 0, -2, 0, 0, -2, 0, 1, 0, 0, 4, 0, 0, 0, 0, -1, -3, 0, -4, 0, -6, 0, 4, 0, 3, 0, 2, -2, 0, -2, 0, 0, 0, 0, 4, -2, 5, 1, 0, -1, 0, -1, -2, 0, 0, 1, 0, -1, 0, 0, 4, 0, 0, 0, 2, -1, 3, 0, 0, 0, 0, 2, 0, 0, 0, -1, 4, 0, 0, 0, 4, 0, -6, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 2, -1, 0, -1, -2, 0, -9, 0, 0, 1, 10, 0, 0, 0, -3, -1, 0, -1, 0, -2, 4, 0, 3, 0, 0, 0, 12, 0, -2, 1, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, -1, 0, 0, -8, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, -1, 0, 0, 0, 0, 6, 0, 2, 0, 0, 0, 4, 0, 3, 0, 0, 0, -4, 0, -4, 0, 0, 0, 0, 0, 7, -1, 0, 0, 3, 0, 0, -2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, -6, 0, 2, 0, 4, 2, 0, 0, 0, 0, 0, -1, -6, 0, 0, 0, 3, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3, 0, -2, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, -2, 0, 0, 0, 1, 0, -2, 8, 0, 0, -1, -3, 1, 0, 1, -5, 0, -2, 0, 0, 4, 2, -2, -4, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -2, 0, -4, 0, -2, 0, 0, -1, 0, 0, 0, -1, -2, 0, -3, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 8, 0, 0, 0, 6, -2, 0, 0, 5, -2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, -6, 0, 2, -2, -3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, -10, 2, 0, 0, 0, 0, 0, 0, 4, -1, 0, 0, 0, 2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, -4, 0, 3, 1, 0, -2, 0, -2, 0, 0, 0, 0, -2, -2, 4, 0, 0, 0, 3, 0, 2, 0, -6, 0, -4, 0, 0, 0, -2, -1, -4, 0, 0, 2, -3, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -6, 0, 14, 0, 0, 0, 0, 0, -2, 0, -4, 0, 0, 2, 0, -1, 0, 0, -5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, -6, -1, -2, -1, -3, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, -1, 0, 0, 0, 2, 4, 1, -6, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, -4, -2, 0, 2, 0, 0, 4, 0, 6, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, -3, 0, -2, -2, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 1, 3, 0, 0, 0, -14, -2, -2, 0, 3, 1, 0, -1, -4, 0, 2, 0, 0, -2, 0, 0, -5, 0, -4, 0, 0, -2, 0, 0, 0, 0, -4, -1, -3, 0, 0, -1, 0, 0, 0, 0, -6, 0, 0, 0, 0, 0, -2, 2, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 0, -1, -2, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 1, -3, 0, 0, 0, 0, 0, -2, -1, 0, 0, 0, 0, 0, 0, -4, -2, 0, 0, 0, 2, -6, 0, -2, 0, 3, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, -5, -1, 0, 0, 6, 2, 0, 0, -8, 0, 0, -1, 0, 2, 0, 0, -7, 0, 0, -2, 0, 0, 0, 0, 4, -1, 0, 0, -3, 0, -2, 0, 0, 0, 0, 1, 0, 0, 0, 0, -4, 0, -2, 0, 0, 1, 4, 0, 0, 2, 2, 2, 0, -2, 2, -1, 0, 0, 0, 2, 0, 0, 0, -1, 10, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 2, 3, 0, 0, 2, 8, 2, 0, 0, -3, -1, 0, 0, -4, 0, 0, 0, 3, -2, 2, 0, 15, 0, -2, 0, -3, 0, 0, 2, 0, -1, 0, 0, 3, 0, 0, -1, 0, 0, -2, 1, -6, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 4, 4, 0, -2, 1, 0, 0, 0, 0, 0, 0, 0, 0, -6, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, -1, 0, 2, 0, 0, 0, 1, 0, -2, 0, -1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, -1, 0, 0, -4, 0, -4, 0, 0, 2, -2, 0, -5, 2, 2, 2, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, -2, 0, -9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 8, 0, -3, 0, 0, 0, 0, 1, 0, 0, 0, 0, -2, 2, -6, 2, 0, 2, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, -2, 1, -5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, -8, -1, 4, 0, 0, 0, 0, 1, -2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -3, 2, 0, 1, -12, -2, 0, 0, 3, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 5, 0, 0, -2, 6, -2, 0, 0, 0, -1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, -2, -2, 4, 0, 0, -4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, -2, -2, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -2, 0, -1, 0, 2, 0, -2, 6, 0, 0, 0, -7, 0, 0, 0, 0, 0, 2, 0, 0, -2, 0, 0, 0, 0, 4, 0, -5, 0, 0, 0, 0, 0, 0, 0, -4, 0, -4, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, -8, 0, -2, 0, -6, 0, 2, 0, 0, 0, 0, 0, 0, -1, 0, -2, 0, 0, -2, 0, -6, 0, 0, 0, -11, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -5, 0, 0, -2, 6, 0, 0, 0, 0, 0, 4, 0, 0, 1, -2, 0, -6, 0, 4, 0, 6, 0, 0, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, -4, 0, 0, 0, -2, 0, 0, 0, 0, -2, 0, 1, 0, 0, 0, 0, 0, 0, 3, -1, -2, 0, 0, 0, 0, 0, 0, 0, -2, -1, 0, -1, 0, 0, -6, 0, -2, 2, 0, 0, -4, -1, 0, -1, 0, 0, -5, 0, -2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 8, -1, -2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0, -1, 0, 1, 0, 0, -2, -2, 0, 0, 0, 0, 0, 1, 0, 0, -3, 0, 0, 0, 0, 1, -2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, -2, 0, -8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, -4, 0, -2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, -2, -2, 0, 0, -1, 0, 0, 3, 0, 8, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 1, 18, 0, 0, -1, 0, 0, 0, 0, 0, 1, -2, 0, 0, 0, 2, 0, 0, 0, 0, -1, 0, 0, -2, 0, 8, 0, 0, 0, 6, 0, 0, 0, 0, -1, 0, 0, 3, 0, 0, -4, 0, -2, 0, 0, -6, -1, 0, 0, 10, 1, -4, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, -7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 1, 0, 0, 0, -1, 2, 0, 4, 0, 4, -2, 0, -2, 0, -1, 0, -1, 0, 0, -3, 2, 0, 1, 8, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -3, 0, -2, 0, 8, -1, 0, 0, 0, 0, 2, 0, 8, 2, 0, -2, 0, 2, 0, 2, 0, 0, -2, 0, -3, 0, 0, 2, 0, 0, 0, 0, 0, 0, -4, 0, 0, 0, -2, 0, -6, 0, 0, 0, 12, 1, 0, -1, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 5, 0, -2, 2, 0, 0, 0, -1, 0, 1, 0, 0, 3, 0, -2, 0, 0, 0, 2, 0, -3, 1, 0, 0, 0, 2, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -4, 0, 0, 0, 6, 3, -2, 1, -12, 0, -2, 0, -6, -1, 0, 0, 0, 1, -4, 1, 3, 0, -2, 0, 0, 0, 0, -1, 0, 0, -2, -2, 4, 0, 0, 0, 0, 0, 0, -1, -10, 0, 2, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, -2, 0, 1, 0, 0, 2, -1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, -1, 0, 0, -2, -1, 0, 0, 0, 0, 0, 0, 0, 2, 10, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 1, -2, -2, 0, 4, 0, 0, 0, 0, 0, -1, 0, -2, -2, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, -4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 8, 0, 0, -1, -6, -2, -2, 0, -5, 2, 2, -1, 6, -1, 0, -1, -4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, -6, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 2, -6, -1, 4, 0, -4, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -2, 0, 4, 0, 0, 0, 3, 2, -2, -1, 0, -1, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, -1, 0, 0, -7, 0, 4, 0, 0, 0, 0, 2, 0, 0, 2, 1, -3, 0, 0, 0, 0, 2, 0, 0, -6, 0, 0, 0, 0, 0, -2, -1, 0, 0, 0, 1, 0, -2, 0, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 4, 0, 0, 1, 0, -2, 0, 2, -2, 0, 3, 1, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 16, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, -2, 0, 0, 0, 0, 0, 1, -3, 1, 0, 0, 0, 0, -4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1, 0, 0, -2, 0, -4, 0, 0, 1, -6, 0, 0, 0, -7, 0, 0, 0, 6, 0, 0, 2, 4, 0, 0, 0, 3, -1, -4, 2, 0, 0, -4, 0, 0, -1, 0, 0, 0, 0, 0, 0, 6, 0, -4, 0, 6, 0, 2, 0, 0, -2, -2, -1, 0, 0, -2, 0, -3, 0, 0, 0, 0, 1, 2, 0, 0, 1, -2, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, -1, 3, 0, -2, 1, 4, 1, 0, 0, 0, -2, 0, 1, 0, 0, 4, 0, 0, -1, 0, -2, 0, -1, -2, 0, 6, 0, 0, 0, 0, -1, 0, 2, 6, 0, 0, -1, -8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, -2, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 3, 0, -2, 0, 0, 0, -2, 0, -3, -1, 0, 0, -12, -1, 0, 0, 0, 0, 2, 0, 12, 0, 0, 0, 0, -2, 0, 0, -4, -1, -2, -1, 0, 0, -2, -1, 0, -1, -4, 1, 6, 0, 0, 1, 4, 0, 0, 0, 0, -1, 0, 0, 13, 0, 0, 0, -3, 0, 2, 0, 0, 0, 0, -3, 0, 1, 0, 2, 0, 0, 0, 0, -3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 2, 4, 0, 4, 0, -2, -1, 0, 0, 0, 0, -10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, -1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, -2, 2, 0, 0, 0, -1, 0, 0, 0, 1, 8, 0, 0, 0, 3, 1, 0, 2, 0, 0, 0, 0, 6, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, -5, 0, 0, 2, -6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 3, -2, 0, 0, -1, 0, 0, 4, 0, 0, 0, 0, 0, -4, -1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, -4, 0, -1, 2, 0, 0, 0, 0, 2, 0, 2, 4, 0, -8, 0, 0, 0, 3, 1, 0, -1, 5, 0, 4, 0, 0, 2, 0, 1, 0, 1, 0, -1, 0, 0, -2, 0, -7, -1, 0, 0, 0, 1, 0, 0, 4, 0, 2, 0, 0, 0, 0, -2, 0, 0, -2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, -4, 0, 0, 1, -3, 1, 0, 0, 0, -1, 0, 0, 0, 1, 8, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, -4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, -4, 0, 10, 0, 0, 0, 0, 0, -2, -1, -4, 0, 2, 0, 0, -2, 2, 0, -6, 0, 0, 0, -3, 0, 0, -1, 0, 0, 8, 0, 3, 0, -4, 0, -5, -1, 0, 0, 0, 0, -2, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, -2, -1, 0, 1, 0, 0, -3, 0, 0, 2, -5, 0, 0, 0, -3, 0, 0, 0, 0, -1, -2, -2, 0, 0, 0, 0, 0, 0, -2, 0, -3, 1, 0, 0, -4, -1, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, -2, -1, 0, 0, 0, -2, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 10, 0, 0, 1, 0, 1, 0, 0, 8, 0, -4, 2, 0, 2, 0, 0, 6, 0, -4, 0, 6, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 4, -2, 0, 0, 0, 0, 0, -1, -7, 0, 0, 2, 3, 0, -4, 2, -4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 3, 0, 0, 2, 0, 0, 0, -2, 0, 0, 4, 1, -12, 0, 4, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 5, 0, -2, 0, 0, -1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, -2, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, -6, 1, 0, 0, 4, 0, 2, 0, 0, 0, -4, -1, 0, 0, -4, 0, 0, 0, 0, -2, 0, 0, 0, -1, -3, 0, -4, 0, 5, -1, 0, -2, 0, 0, 0, -1, 4, 0, 0, 0, 3, 0, 0, 0, -7, -1, 0, 0, -6, 0, 0, -1, 0, 0, 0, 0, -3, 0, -2, 0, -5, 0, 0, 0, 0, 2, 4, 0, -4, -1, 0, 4, -3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0, 2, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, -9, 0, 0, 0, 16, 1, 0, 1, 0, -2, 0, 0, 0, 0, 0, 0, -3, 1, 0, -1, -5, 0, 0, 0, 0, 1, 4, -1, 0, 0, -2, 0, 0, 0, 0, 2, -6, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 2, 0, 0, -6, 0, 0, 0, -7, 0, -2, 1, -3, 0, -2, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, -2, 2, 0, -1, 0, 0, 0, 0, 0, -1, 0, -1, -2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 6, 0, 0, 2, 0, 1, -2, 0, 0, 0, 0, 0, 0, 1, 4, 0, 6, 0, 2, 0, -8, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, -1, 0, 1, -5, 0, -4, 0, 0, -1, 4, -1, 0, 0, 0, 0, 6, 0, 0, -2, 7, 0, 0, -1, 0, 2, 4, 0, 0, -1, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, -4, -4, 0, 0, -3, 0, -4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, -2, 0, 0, 0, -1, 10, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, -1, 0, 0, 0, 0, -16, -1, -4, 0, -3, 0, 0, -1, 4, 0]

        # Precomputations
        if prev != N:
            print('Starting precomputations 1/2, Time:', datetime.now().time())
            prc1 = [0] + [-RR(Ei(RF(-2*pi*n/sqrt(N)))) for n in range(1, coefs)]
            print('Starting precomputations 2/2, Time:', datetime.now().time())
            prc2 = [-RR(integral(exp(-2*pi*n*x), x)(x = RR(1/sqrt(N)))) for n in range(coefs)]
            prev = N
        else:
            print('Precomputations not necessary')

        # Find L-value 
        print('Starting L-value, Time:', datetime.now().time())
        cf_g = [[G[j](cf[i]) for i in range(len(cf))] for j in range(len(G))]
        cf_g_ev = [[eval(str(co).replace('^', '**'), {var: CC(sum(coef*x**i for i, coef in enumerate(poly_l)).roots(ring=QQbar)[0][0])}) for co in g_line] for g_line in cf_g]
        beta = [evaluate_W(cf_g_ev[j], 1e-6, RR(1/sqrt(N)))/evaluate([conjugate(cf_g_ev[j][i]) for i in range(len(cf_g_ev[j]))], 1e-6, RR(1/sqrt(N))) for j in range(len(G))]
        cWf_g = [[beta[j]*CC(conjugate(cf_g_ev[j][i])) for i in range(len(cf))] for j in range(len(G))]
        l_vals = [lfunction_value(cf_g_ev[i], cWf_g[i], N, prc1 = prc1, prc2 = prc2) for i in range(len(G))]
        l_val = sum([CC(G[i](b))*l_vals[i] for i in range(len(G))])

        # Find polynomial
        print('Finding polynomial, Time:', datetime.now().time())
        pol = find_polynomial(exp(l_val), [5, 10, 15, 20, 25, 30, 35, 40, 45, 50], 200)
        print('Polynomial: ', pol)

        # Check that it is not done
        results = pd.read_csv('res.txt', sep=':', index_col=0)
        results_exp = pd.read_csv('res_exp.txt', sep=':', index_col=0)
        # if results.loc[index, 'Stark unit, b=1'] != '-':
        #     continue

        # Save results
        # if results.loc[index, 'Stark unit, b=1'] in ['-', 'error', 'Mod problem']:
        #     results.loc[index, 'Stark unit, b=1'] = pol
        #     results.to_csv("res.txt", sep=':')
        # if results_exp.loc[index, 'Stark unit, b=1'] in ['-', 'error', 'Mod problem']:
        #     results_exp.loc[index, 'Stark unit, b=1'] = pol
        #     results_exp.loc[index, 'L-value'] = RF(exp(l_val).real())
        #     results_exp.loc[index, 'precision L-value'] = pres_prec
        #     results_exp.loc[index, 'first coefficients'] = str(coefs_str[:10])
        #     results_exp.to_csv("res_exp.txt", sep=':')

        results.loc[index, 'Stark unit, b=1'] = pol
        results.to_csv("res.txt", sep=':')

        results_exp.loc[index, 'Stark unit, b=1'] = pol
        results_exp.loc[index, 'L-value'] = RF(exp(l_val).real())
        results_exp.loc[index, 'precision L-value'] = pres_prec
        results_exp.loc[index, 'first coefficients'] = str(cf[:10])
        results_exp.to_csv("res_exp.txt", sep=':')

        print('End. Time:', datetime.now().time(), '\n')
    except Exception as e:
        results_exp = pd.read_csv('res_exp.txt', sep=':', index_col=0)
        if results_exp.loc[index, 'Stark unit, b=1'] == '-':
            results_exp.loc[index, 'Stark unit, b=1'] = 'error'
            results_exp.to_csv("res_exp.txt", sep=':')
        print('ERROR:', e, '\n')

STARTING 14976 , N =  3407 , Label  3407.1.b.a , Time: 23:47:03.413654
Presumed precision:  233.8 digits
Starting coefficients, Time: 23:47:03.566809
Starting Galois group, Time: 23:47:03.566876
The Galois group is Galois group 1T1 (S1) with order 1 of x
Starting precomputations 1/2, Time: 23:47:03.568113
Starting precomputations 2/2, Time: 23:49:24.520549
Starting L-value, Time: 23:49:32.646355
Finding polynomial, Time: 23:49:39.070292
Polynomial:  -
End. Time: 23:49:42.027753 

